# Pouch Cell Model Parameter Identification

In this notebook, we present the single particle model with a two dimensional current collector. This is achieved via the potential-pair models introduced in Marquis et al. [[1]](https://doi.org/10.1149/1945-7111/abbce4) as implemented in PyBaMM. At a high-level this is accomplished as a potential-pair model which is resolved across the discretised spatial locations.

### Importing libraries

If you don't already have PyBOP installed, check out the [installation guide](https://pybop-docs.readthedocs.io/en/latest/installation.html) first.

We begin by importing the necessary libraries.

In [ ]:
import numpy as np
import pybamm

import pybop

go = pybop.plot.PlotlyManager().go
pybop.plot.PlotlyManager().pio.renderers.default = "notebook_connected"

## Generating Synthetic Data

To demonstrate parameter estimation, we first need some data. We will generate synthetic data using a forward model, which requires defining the parameter values and the model itself.

We start by creating an example parameter values and then instantiate the single-particle model (SPM):

In [ ]:
parameter_values = pybamm.ParameterValues("Marquis2019")
parameter_values.update(
    {
        "Negative electrode active material volume fraction": 0.495,
        "Positive electrode active material volume fraction": 0.612,
    }
)
model = pybamm.lithium_ion.SPM(
    options={"current collector": "potential pair", "dimensionality": 2},
)

Let's also update the number of spatial grid points for simulation of the potential-pair model.

In [ ]:
var_pts = {
    "x_n": 10,  # negative electrode
    "x_s": 10,  # separator
    "x_p": 10,  # positive electrode
    "y": 5,  # y direction
    "z": 5,  # z direction
    "r_n": 10,  # negative particle radius
    "r_p": 10,  # positive particle radius
}

We can then simulate the model using the default constant discharge current to generate voltage data. To make the parameter estimation more realistic, we add Gaussian noise to the data.

In [ ]:
t_eval = np.arange(0, 900, 3)
sim = pybamm.Simulation(model, parameter_values=parameter_values, var_pts=var_pts)
sol = sim.solve(t_eval=t_eval, initial_soc=0.5)
voltage = sol["Voltage [V]"](t_eval)
current = sol["Current [A]"](t_eval)

sigma = 0.001  # 1 mV
corrupt_values = voltage.data + np.random.normal(0, sigma, len(t_eval))

The dataset for optimisation is composed of time, current, and the noisy voltage data:

In [ ]:
dataset = pybop.Dataset(
    {
        "Time [s]": t_eval,
        "Current function [A]": current,
        "Voltage [V]": corrupt_values,
    }
)

## Identifying the Parameters

To set up the parameter estimation process, we select the parameters for estimation and set up their prior distributions and bounds.

In [ ]:
parameters = [
    pybop.Parameter(
        "Negative electrode active material volume fraction",
        prior=pybop.Gaussian(0.7, 0.05),
        bounds=[0.45, 0.9],
    ),
    pybop.Parameter(
        "Positive electrode active material volume fraction",
        prior=pybop.Gaussian(0.58, 0.05),
        bounds=[0.5, 0.8],
    ),
]

With the dataset and parameters defined, we can set up the optimisation problem and the optimiser.

In [ ]:
builder = (
    pybop.builders.Pybamm()
    .set_dataset(dataset)
    .set_simulation(
        model, parameter_values=parameter_values, solver=pybamm.IDAKLUSolver()
    )
    .add_cost(pybop.costs.pybamm.SumSquaredError("Voltage [V]", "Voltage [V]"))
)
for param in parameters:
    builder.add_parameter(param)
problem = builder.build()

options = pybop.PintsOptions(max_iterations=30)
optim = pybop.CMAES(problem, options=options)

We proceed to run the CMA-ES optimisation algorithm to estimate the parameters. After the optimisation, we can examine the estimated parameter values:

In [ ]:
results = optim.run()
results.x

array([0.45000026, 0.5000001 ])

## Plotting and Visualisation

PyBOP provides various plotting utilities to visualise the results of the optimisation.

### Spatial Plotting

We can now plot the spatial variables from the solution object. First, the negative current collector potential at the final time can be displayed. In this example, this is just a reference variable, but could be used for fitting or optimisation in the correct workflows.

In [ ]:
sim = pybamm.Simulation(
    model, parameter_values=results.parameter_values, var_pts=var_pts
)
sol = sim.solve(t_eval=t_eval, initial_soc=0.5)

go.Figure(
    [
        go.Contour(
            x=sol["y [m]"].data[:, 0, -1],
            y=sol["z [m]"].data[0, :, -1],
            z=sol["Negative current collector potential [V]"].data[:, :, -1],
            colorscale="Viridis",
        )
    ],
    layout=dict(
        title="Negative current collector potential [V]",
        xaxis_title="y / m",
        yaxis_title="z / m",
        width=600,
        height=600,
    ),
)

We can also plot the positive current collector potential at the final time.

In [ ]:
go.Figure(
    [
        go.Contour(
            x=sol["y [m]"].data[:, 0, -1],
            y=sol["z [m]"].data[0, :, -1],
            z=sol["Positive current collector potential [V]"].data[:, :, -1],
            colorscale="Viridis",
        )
    ],
    layout=dict(
        title="Positive current collector potential [V]",
        xaxis_title="y / m",
        yaxis_title="z / m",
        width=600,
        height=600,
    ),
)

### Convergence and Parameter Trajectories

To assess the optimisation process, we can plot the convergence of the cost function and the trajectories of the parameters:

In [ ]:
pybop.plot.convergence(optim)
pybop.plot.parameters(optim);

### Cost Landscape

Finally, we can visualise the cost landscape and the path taken by the optimiser:

In [ ]:
pybop.plot.surface(optim);